In [ ]:
%load_ext autoreload
%autoreload 2
import time, base64,ssl, os, re,json5, pprint,random , math, hashlib, inspect
import pandas as pd
from selenium import webdriver# Controls the browser
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By                   # Locators (ID, CLASS_NAME, XPATH, etc.)
from selenium.webdriver.support.ui import WebDriverWait       # Waits for elements to appear
from selenium.webdriver.support import expected_conditions as EC  # Conditions like "visible", "clickable"
from selenium.webdriver.chrome.options import Options
from urllib.parse import urljoin
from bs4 import BeautifulSoup
from app.utils import Helper
from app.operation_executor import OperationExecutor
import pandas as pd
import numpy as np

from io import StringIO

In [ ]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
wait = WebDriverWait(driver, 10)

# Switch to iframe
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

# === Just print dropdown 3 options ===
for op1 in dropdown_1:
    dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
    select_dropdown(dropdowns[0], op1)

    for op2 in dropdown_2:
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[1], op2)

        # Open dropdown 3 and collect options
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        arrow = dropdowns[2].find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
        arrow.click()
        time.sleep(0.5)

        options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
        op3_options = [opt.text.strip() for opt in options]

        arrow.click()  # close dropdown 3

        print(f"{op1} + {op2} → {op3_options}")

driver.quit()


In [ ]:
# import time
# import itertools
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os, time, itertools, glob
from selenium.webdriver.chrome.options import Options

download_dir = r"C:\Users\kaustubh.keny\Downloads"

chrome_options = Options()
chrome_options.add_experimental_option("prefs", {
    "download.default_directory": download_dir,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True
})

driver = webdriver.Chrome(options=chrome_options)

# Setup
driver = webdriver.Chrome()
driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
wait = WebDriverWait(driver, 10)

# Dropdown values
dropdown_1 = ['Open Ended', 'Close Ended']
dropdown_2 = ['Equity', 'Debt', 'Hybrid', 'Solution Oriented', 'Other']
dropdown_3 = ['Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap', 'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused', 'Sectoral / Thematic']
dropdown_4_value = 'All'
combinations = list(itertools.product(dropdown_1, dropdown_2, dropdown_3))

# Utility functions
def wait_for_iframe(timeout=30):
    for _ in range(timeout):
        try:
            iframe = driver.find_element(By.TAG_NAME, "iframe")
            return iframe
        except:
            time.sleep(1)
    raise Exception("Iframe not found after waiting")

def select_dropdown(dropdown_element, value):
    arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
    arrow.click()
    time.sleep(0.5)
    options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
    for opt in options:
        if opt.text.strip() == value:
            opt.click()
            break
    time.sleep(0.5)

def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        const input = arguments[0];
        const nativeInputValueSetter = Object.getOwnPropertyDescriptor(window.HTMLInputElement.prototype, 'value').set;
        nativeInputValueSetter.call(input, arguments[1]);
        input.dispatchEvent(new Event('input', { bubbles: true }));
        input.dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)


def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(3)

def download_excel():
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()
    time.sleep(2)

def rename_latest_excel(combo):
    time.sleep(3)  # wait for download to finish
    files = glob.glob(os.path.join(download_dir, "*.xlsx"))
    latest_file = max(files, key=os.path.getctime)

    original_name = os.path.basename(latest_file).replace(".xlsx", "")
    combo_str = "_".join(combo).replace(" ", "-")

    new_name = f"{original_name}_{combo_str}.xlsx"
    new_path = os.path.join(download_dir, new_name)

    os.rename(latest_file, new_path)
    print(f"✅ Renamed to: {new_name}")

for idx, combo in enumerate(combinations):
    try:
        print(f"🔄 Combo {idx+1}: {combo}")

        iframe = wait_for_iframe()
        driver.switch_to.frame(iframe)

        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[0], combo[0])
        select_dropdown(dropdowns[1], combo[1])
        select_dropdown(dropdowns[2], combo[2])
        select_dropdown(dropdowns[3], dropdown_4_value)

        set_date("31-Jul-2025")
        click_go()
        download_excel()
        print("📥 Download triggered — waiting for file...")
        rename_latest_excel(combo)

        driver.switch_to.default_content()

    except Exception as e:
        print(f"⚠️ Skipping combo {combo} due to error: {e}")
        driver.switch_to.default_content()
        continue

driver.quit()

In [5]:
import os
import time
import json
import shutil
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
# -----------------------
# Setup
# -----------------------
DOWNLOAD_DIR = os.path.join(os.getcwd(), "mf_downloads")
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

chrome_options = webdriver.ChromeOptions()
prefs = {
    "download.default_directory": DOWNLOAD_DIR,
    "download.prompt_for_download": False,
    "download.directory_upgrade": True,
    "safebrowsing.enabled": True,
}
chrome_options.add_experimental_option("prefs", prefs)

driver = webdriver.Chrome(options=chrome_options)
wait = WebDriverWait(driver, 10)

driver.get("https://www.amfiindia.com/research-information/other-data/mf-scheme-performance-details")
iframe = wait.until(EC.presence_of_element_located((By.TAG_NAME, "iframe")))
driver.switch_to.frame(iframe)

# -----------------------
# Dropdown data
# -----------------------
dropdown_1 = ['Open Ended', 'Close Ended']
op3_map = {
    "Equity": [
        'Large Cap', 'Large & Mid Cap', 'Flexi Cap', 'Multi Cap', 'Mid Cap',
        'Small Cap', 'Value', 'ELSS', 'Contra', 'Dividend Yield', 'Focused',
        'Sectoral / Thematic'
    ],
    "Debt": [
        'Long Duration', 'Medium to Long Duration', 'Short Duration', 'Medium Duration',
        'Money Market', 'Low Duration', 'Ultra Short Duration', 'Liquid', 'Overnight',
        'Dynamic Bond', 'Corporate Bond', 'Credit Risk', 'Banking and PSU', 'Floater',
        'FMP', 'Gilt', 'Gilt with 10 year constant duration'
    ],
    "Hybrid": [
        'Aggressive Hybrid', 'Conservative Hybrid', 'Equity Savings', 'Arbitrage',
        'Multi Asset Allocation', 'Dynamic Asset Allocation or Balanced Advantage',
        'Balanced Hybrid'
    ],
    "Solution Oriented": ["Children's", 'Retirement'],
}
dropdown_4_value = "All"
DATE_STR = "31-Jul-2025"

# -----------------------
# Generate combos.json if missing
# -----------------------
if not os.path.exists("combos.json"):
    all_combos = []
    for op1 in dropdown_1:
        for op2, op3_list in op3_map.items():
            for op3 in op3_list:
                all_combos.append({
                    "op1": op1,
                    "op2": op2,
                    "op3": op3,
                    "op4": dropdown_4_value
                })

    with open("combos.json", "w", encoding="utf-8") as f:
        json.dump(all_combos, f, indent=2)

with open("combos.json", "r", encoding="utf-8") as f:
    combos = json.load(f)

# -----------------------
# Helpers
# -----------------------
def select_dropdown(dropdown_element, value, retries=3):
    for attempt in range(retries):
        try:
            close_overlays()  # 🛑 ensure previous overlay is gone
            arrow = dropdown_element.find_element(By.CSS_SELECTOR, ".ng-arrow-wrapper")
            arrow.click()
            time.sleep(0.5)
            options = wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".ng-option")))
            for opt in options:
                if opt.text.strip() == value:
                    opt.click()
                    return
            raise ValueError(f"Option '{value}' not found")
        except (StaleElementReferenceException, TimeoutException) as e:
            print(f"⚠️ Retrying select for '{value}' ({attempt+1}/{retries}) due to {e.__class__.__name__}")
            time.sleep(1)
    raise Exception(f"Failed to select '{value}' after retries")


def set_date(date_str):
    date_input = driver.find_element(By.CSS_SELECTOR, "input[bsdatepicker]")
    driver.execute_script("""
        arguments[0].value = arguments[1];
        arguments[0].dispatchEvent(new Event('input', { bubbles: true }));
        arguments[0].dispatchEvent(new Event('change', { bubbles: true }));
    """, date_input, date_str)
    time.sleep(1)

def click_go():
    go_button = driver.find_element(By.CSS_SELECTOR, "button[mat-raised-button]")
    go_button.click()
    time.sleep(3)

def download_excel(custom_name):
    excel_icon = driver.find_element(By.CSS_SELECTOR, ".excel-border.cursor-pointer")
    excel_icon.click()

    file_path = None
    for _ in range(30):  # wait up to 30s
        files = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith(".xls") or f.endswith(".xlsx")]
        if files:
            file_path = os.path.join(DOWNLOAD_DIR, files[0])
            break
        time.sleep(1)

    if file_path:
        new_path = os.path.join(DOWNLOAD_DIR, custom_name + ".xlsx")
        shutil.move(file_path, new_path)
        print(f"✅ Saved: {new_path}")
    else:
        print("⚠️ Download failed or file not found.")

def handle_popup(timeout=5):
    try:
        alert = WebDriverWait(driver, timeout).until(EC.alert_is_present())
        print("⚠️ Popup detected:", alert.text)
        alert.accept()   # closes the popup
        return True
    except TimeoutException:
        return False


def close_overlays():
    """Try to close any open dropdown overlays"""
    try:
        body = driver.find_element(By.TAG_NAME, "body")
        body.send_keys(Keys.ESCAPE)  # press ESC
        time.sleep(0.5)
    except Exception:
        pass


# -----------------------
# Execution loop
# -----------------------
for combo in combos:
    try:
        op1, op2, op3, op4 = combo["op1"], combo["op2"], combo["op3"], combo["op4"]

        # reset dropdown elements each loop
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[0], op1)
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[1], op2)
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[2], op3)
        dropdowns = driver.find_elements(By.CSS_SELECTOR, "ng-select")
        select_dropdown(dropdowns[3], op4)

        print(f"▶️ Running combo: {op1}, {op2}, {op3}, {op4}")
        set_date(DATE_STR)
        click_go()

        if handle_popup():
            print(f"⏭️ Skipping combo (no data): {op1}, {op2}, {op3}")
            continue

        safe_name = f"FundPerformance-{op1}_{op2}_{op3}_{DATE_STR}".replace(" ", "_").replace("/", "-")
        download_excel(safe_name)

    except Exception as e:
        print(f"[ERROR] {combo}: {e}")

driver.quit()


The chromedriver version (139.0.7258.138) detected in PATH at C:\Users\rando\chromedriver-win64\chromedriver.exe might not be compatible with the detected chrome version (140.0.7339.81); currently, chromedriver 140.0.7339.80 is recommended for chrome 140.*, so it is advised to delete the driver in PATH and retry


▶️ Running combo: Open Ended, Equity, Large Cap, All
✅ Saved: c:\Users\rando\Office Projects\selenium-scrape\mf_downloads\FundPerformance-Open_Ended_Equity_Large_Cap_31-Jul-2025.xlsx
▶️ Running combo: Open Ended, Equity, Large & Mid Cap, All
✅ Saved: c:\Users\rando\Office Projects\selenium-scrape\mf_downloads\FundPerformance-Open_Ended_Equity_Large_&_Mid_Cap_31-Jul-2025.xlsx
▶️ Running combo: Open Ended, Equity, Flexi Cap, All
✅ Saved: c:\Users\rando\Office Projects\selenium-scrape\mf_downloads\FundPerformance-Open_Ended_Equity_Flexi_Cap_31-Jul-2025.xlsx
▶️ Running combo: Open Ended, Equity, Multi Cap, All
✅ Saved: c:\Users\rando\Office Projects\selenium-scrape\mf_downloads\FundPerformance-Open_Ended_Equity_Multi_Cap_31-Jul-2025.xlsx
▶️ Running combo: Open Ended, Equity, Mid Cap, All
✅ Saved: c:\Users\rando\Office Projects\selenium-scrape\mf_downloads\FundPerformance-Open_Ended_Equity_Mid_Cap_31-Jul-2025.xlsx
▶️ Running combo: Open Ended, Equity, Small Cap, All
✅ Saved: c:\Users\rando\

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
from urllib.parse import urljoin
import os
ops = OperationExecutor()
pathList = [r"25-09-03T16-21-30_cache.json",r"25-09-04T16-02-03_cache.json"]

for path in pathList:
    
    root_file = os.path.basename(path).split("_")[0]
    function_to_execute = {
    "primary":[["normalize_df","value","norm_table","table_html"],["sha1","value","SHA_ONE","pdf"]],
    "secondary":[["sha1","norm_table","SHA_ONE"]],
    }
    try:
        data = Helper.load_json(path, typ="json5")
        processed_data = ops.runner(data,function_to_execute)
        Helper.save_json(processed_data,f"{root_file}_process.json")
    except Exception as e:
        print(f"\nError in Processing.. Skipping, {e}")
        continue

In [ ]:
from app.operation_executor import OperationExecutor
from app.utils import Helper
ops = OperationExecutor()
yesterday = Helper.load_json("25-09-03T16-21-30_process.json")
today = Helper.load_json("25-09-04T16-02-03_process.json")

result = ops.process_comparison(yesterday,today,key="SHA_ONE")
Helper.save_json(result, "compare-030925.json")

ops.generate_sorted_excel_report(result)

In [ ]:
"Indian Bank"

link1 = "https://www.indianbank.in/departments/fcnr-b-deposits/"
ink2 = "https://www.indianbank.in/departments/interest-rates-for-deposits-of-above-rs-5-00crore/"

In [ ]:
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
import random, time, math, ssl

ssl._create_default_https_context = ssl._create_stdlib_context

def human_scroll(driver, total_scroll=2000, step=50):
    for y in range(0, total_scroll, step):
        driver.execute_script(f"window.scrollTo(0, {y});")
        time.sleep(random.uniform(0.05, 0.3))

def human_move_mouse(driver, start=(0, 0), end=(300, 300), steps=30):
    actions = ActionChains(driver)
    x1, y1 = start
    x2, y2 = end
    for t in range(steps):
        progress = t / steps
        x = x1 + (x2 - x1) * progress + random.uniform(-2, 2)
        y = y1 + (y2 - y1) * progress + math.sin(progress * math.pi) * 20
        actions.move_by_offset(int(x), int(y))
    actions.perform()

def human_click(elem):
    time.sleep(random.uniform(0.2, 1.5))
    elem.click()

profile_path = Helper.load_json("paths.json").get("profile_path")
profile_dir = Helper.load_json("paths.json").get("profile_name")

options = Options()
options.add_argument(f"--user-data-dir={profile_path}")
options.add_argument(f"--profile-directory={profile_dir}")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--start-maximized")

driver = uc.Chrome(options=options)

driver.get("https://www.unionbankofindia.co.in/en/details/rate-of-interest")

human_scroll(driver, total_scroll=1500, step=70)
human_move_mouse(driver, start=(0, 0), end=(200, 200), steps=40)

try:
    table = driver.find_element(By.TAG_NAME, "table")
    human_click(table) # Just demo click
    print("Table found:", table.get_attribute("outerHTML")[:200])
except Exception as e:
    print("No table found:", e)

time.sleep(5)
driver.quit()


In [ ]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time

path = "https://www.nseindia.com/market-data/equity-derivatives-watch"
options = Options()
driver = webdriver.Chrome(options=options)
driver.get(path)

time.sleep(5)

select_element = driver.find_element(By.CSS_SELECTOR, "select.no-border-radius")
select = Select(select_element)

for option in select.options:
    try:
        value = option.get_attribute("value")
        print(f"Selecting: {option.text} ({value})")
        
        select.select_by_value(value)
        time.sleep(2)

        tables = driver.find_elements(By.CSS_SELECTOR, "#eqderivativesTable")
        print(f"Found {len(tables)} tables for {option.text}")
        
        for idx, table in enumerate(tables):
            content = table.get_attribute("outerHTML")
            with open(f"{value}_table_{idx}.html", "w+", encoding="utf-8") as f:
                f.write(content)

    except Exception as e:
        print(f"Failed to select {option.text}: {e}")from app.operation_executor import OperationExecutor

# path = "https://www.jkbank.com/interest-rates"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".owl-nav-carousel li a")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
        
        
# path = "https://www.icicibank.com/personal-banking/deposits/fixed-deposit/fd-interest-rates?ITM=nli_cms_FD_rates_menu_navigation"
# options = Options()
# driver = webdriver.Chrome(options=options)

# driver.get(path)
# tabList = driver.find_elements(By.CSS_SELECTOR,".tabs-list button")
# print(f"Total tabs found: {len(tabList)}")
# for tab in tabList:
#     try:
#         driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", tab)
#         time.sleep(1) 

#         ActionChains(driver).move_to_element(tab).perform()

#         WebDriverWait(driver, 10).until(EC.element_to_be_clickable(tab))
#         print(tab.get_attribute("innerText"))
#         tab.click()
#         time.sleep(1)
        
#         tables = driver.find_elements(By.TAG_NAME, "table")
#         # for idx,table in enumerate(tables):
#         #     content = table.get_attribute("outerHTML")
#         #     with open(f"table_{idx}.html","w+",encoding="utf-8") as f:
#         #         f.write(content)
#     except Exception as e:
#         print(f"Failed to click tab: {e}")
